In [ ]:
from os import getcwd
import os.path as op
from glob import glob

In [ ]:
from clients.utils import create_allocation_table
from clients.expt_recipes.db_query import create_plates_from_allocation_table

expt_name = 'A81_E217'
allocation_table = create_allocation_table(expt_name)
expt_plates = create_plates_from_allocation_table(allocation_table)

In [ ]:
from hardware.qpcr import QpcrDataFile

id_qpcr_plate = 'A81_E217_1'
cwd = getcwd()
nav_root = op.abspath(op.join(cwd, op.pardir))
data_folder = op.join(nav_root, 'vanilla', 'tests', 'data')
f = op.join(data_folder, id_qpcr_plate + '.xls')

qpcr_data = QpcrDataFile(f)
id_qpcr_inst_plate = qpcr_data.get_data_by_well()

In [ ]:
from hardware.labchip import LabChip

lc_plate = '20180103_B'
fds = glob(op.join(data_folder, '*' + lc_plate + '*.csv'))

files = {}
for f in fds:
    if 'PeakTable' in f:
        files['peak_file'] = f
    elif 'RawTable' in f:
        files['raw_file'] = f
    elif 'WellTable' in f:
        
        files['well_file'] = f
lc = LabChip(**files)
lc_inst_plate = lc.get_data_by_well()

In [ ]:
from clients.expt_recipes.db_query import get_assay_amplicon_length
from clients.expt_recipes.vanilla.model_builders import build_qpcr_constituents
from clients.expt_recipes.lost import build_labchip_datas_from_inst_data
from clients.expt_recipes.vanilla.constituents import IdConstituents
from clients.expt_recipes.vanilla.model_builders import (build_id_qpcr_datas_from_inst_data,
                                                         get_wells_by_id_assay)
from clients.expt_recipes.db_query import create_qpcr_lc_mapping, get_lc_dilutions
from clients.expt_recipes.vanilla.models import VanillaMasterTable

In [ ]:
%%time
id_plate_constituents = build_qpcr_constituents(expt_plates[id_qpcr_plate])

wells_by_id_assay = get_wells_by_id_assay(id_plate_constituents)

id_qpcr_datas = build_id_qpcr_datas_from_inst_data(id_plate_constituents, id_qpcr_inst_plate)

In [ ]:
%%time

assays = {}
for assay in wells_by_id_assay:
    for a in assay:
        assays[a] = int(get_assay_amplicon_length(a))

mapping = create_qpcr_lc_mapping(expt_plates[lc_plate])
dilutions = get_lc_dilutions(expt_plates[lc_plate])
lc_datas = build_labchip_datas_from_inst_data(
    id_plate_constituents, lc_inst_plate, mapping, assays, dilutions)

In [ ]:
%%time
qwells = wells_by_id_assay[('Kp_KPC_5.x_KPC19_KPC24',)]
lcwells = []
for qw in qwells:
    if qw in mapping:
        lcwells.append(mapping[qw])
    else:
        lcwells.append(None)

nmt = VanillaMasterTable.create_from_models(qwells,
                                           id_qpcr_plate,
                                           lcwells,
                                           lc_plate,
                                           id_plate_constituents,
                                           id_qpcr_datas,
                                           lc_datas)

In [ ]:
nmt.rows[0]

In [ ]:
import pandas as pd
df = pd.DataFrame(nmt.rows)
df.loc[:, ['qPCR Well', 'Ct', '∆NTC Ct', 'Ct Call', 'Tm Specif','Tm NS', 'Tm PD']]

In [ ]:
grps = {}

for r in nmt.rows:
    grp_key = (r['ID Template Name'], r['ID Template Conc.'],
               r['ID Human Name'], r['ID Human Conc.'])
    grps.setdefault(grp_key, []).append(r)

In [ ]:
from clients.expt_recipes.vanilla.models import VanillaSummaryRow

In [ ]:
VanillaSummaryRow.create_from_master_table_rows(
    grps[(('K.pneumoniae-KPC_1705',), (100.0,), (), ())])